In [18]:
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas

In [19]:
start_date='2001-02-28'
end_date=  '2007-11-30'


datalist={
'S&P 500 Historical- Daily':                     ['new_sandp_500','sp500_df']
,'CBOE Volatility Index (VIX)':                  ['vix_clean','vix_df']
,'Bonds: Treasury Yield 10 Years (TNX)':         ['tnx_clean','tnx_df']
,'Gold: LBMA Gold Price':                        ['lmba_gold_clean','gold_df']
,'US Dollar/USDX Index (DX-Y.NYB)':              ['usd_index_clean','usd_df']
,'Technology Sector: SPDR Fund (XLK) 1998':      ['technology_sector_clean','tech_df']
,'Financial Sector: SPDR Fund (XLF) 1998':       ['financial_sector_clean','fin_df']
,'Energy Sector: SPDR Fund (XLE) 1998':          ['energy_sector_clean','engy_df']
,'Utilities Sector: SPDR Fund (XLU) 1998':       ['utilities_sector_clean','utils_df']
,'Health Care Sector: SPDR Fund (XLV) 1998':     ['healthcare_sector_clean','health_df']
,'US Real Estate ETF: iShares (IYR)':            ['real_estate_etf_clean','prop_df']
,'Oil Prices: Crude Oil WTI Futures':            ['oil_df','oil_df']
}

df_list =[]

for items in datalist.items():
    z=list(items)
    if items[1][1] == 'gold_df':
        # print(f'Working on {items[1][1]}')
        filename="../Resources/clean-data/"+z[1][0]+".csv"
        z[1][1]=pd.read_csv(Path(filename),parse_dates=['Date'],infer_datetime_format=True)
        z[1][1]=z[1][1].loc[(z[1][1]['Date']>=start_date) & (z[1][1]['Date']<=end_date)].sort_values('Date')
        z[1][1]=z[1][1].drop(['Open'],axis=1) #Original data for gold is different than others
        z[1][1]=z[1][1].rename({'Close':'Adj Close'},axis=1) #Original data for gold is different than others
        z[1][1]=z[1][1].drop_duplicates() #Original data for gold is different than others
        z[1][1]=z[1][1].set_index('Date')
        # display(z[1][1].info())
        # display(z[1][1].head())
        # display(z[1][1].tail())
        df_list.append(z[1][1])
    else:
        filename="../Resources/clean-data/"+z[1][0]+".csv"
        # print(f'Working on {items[1][1]}')
        z[1][1]=pd.read_csv(Path(filename),parse_dates=['Date'],infer_datetime_format=True)
        z[1][1]=z[1][1].loc[(z[1][1]['Date']>=start_date) & (z[1][1]['Date']<=end_date)].sort_values('Date')
        z[1][1]=z[1][1].drop(['Open','High','Low','Volume','Close'],axis=1)
        z[1][1]=z[1][1].drop_duplicates()
        z[1][1]=z[1][1].set_index('Date')
        # display(z[1][1].info())
        # display(z[1][1].head())
        # display(z[1][1].tail())
        df_list.append(z[1][1])
   


In [20]:
df_list_new=df_list.copy()

In [21]:
concat_df=pd.DataFrame(df_list_new)
concat_df=pd.concat(df_list_new,axis=1,join='inner')

concat_df.columns.values[0] ='sp500_close'
concat_df.columns.values[1] ='vix_close'
concat_df.columns.values[2] ='tnx_close'
concat_df.columns.values[3] ='gold_close'
concat_df.columns.values[4] ='usd_close'
concat_df.columns.values[5] ='tech_close'
concat_df.columns.values[6] ='fin_close'
concat_df.columns.values[7] ='engy_close'
concat_df.columns.values[8] ='utils_close'
concat_df.columns.values[9] ='health_close'
concat_df.columns.values[10]='prop_close'
concat_df.columns.values[11]='oil_close'

concat_df.head()

d:\Python\envs\dev\lib\site-packages\pandas\core\internals\construction.py:568: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


,sp500_close,vix_close,tnx_close,gold_close,usd_close,tech_close,fin_close,engy_close,utils_close,health_close,prop_close,oil_close
Date,,,,,,,,,,,,
2001-02-28,1239.94,28.350000,4.908,266.70,112.010002,21.326403,14.233952,17.997936,14.725676,20.409981,15.488563,27.42
2001-03-01,1241.23,28.080000,4.856,266.35,111.279999,21.927896,14.249502,18.202068,14.564467,20.338774,15.511208,27.65
2001-03-02,1234.18,27.430000,4.943,263.10,111.169998,21.166521,14.270246,18.616014,14.601309,20.374378,15.686162,27.84
2001-03-05,1241.41,27.120001,4.963,262.50,111.540001,21.623344,14.197648,18.718079,14.573677,20.260431,15.663527,28.60
2001-03-06,1253.80,25.889999,4.971,261.85,111.199997,22.537006,14.239131,18.814478,14.555249,20.331657,15.610002,28.30


In [22]:
start_date='03-01-2001'
end_date='11-30-2001'

RecessionName="9/11 Recession"

recess_sp500  =concat_df.loc[start_date:end_date,:].hvplot(y='sp500_close'  ,height=300,width=600,ylabel='Index Value'                      ,title=f'{RecessionName}: Feb 2020- Apr 2020:S&P 500')
recess_vix    =concat_df.loc[start_date:end_date,:].hvplot(y='vix_close'    ,height=300,width=600,ylabel='Index Value'                      ,title=f'{RecessionName}: Feb 2020- Apr 2020:CBOE Volatility')
recess_tnx    =concat_df.loc[start_date:end_date,:].hvplot(y='tnx_close'    ,height=300,width=600,ylabel='Yield'                            ,title=f'{RecessionName}: Feb 2020- Apr 2020:Treasury Yield 10 Years')
recess_gold   =concat_df.loc[start_date:end_date,:].hvplot(y='gold_close'   ,height=300,width=600,ylabel='Gold price'                       ,title=f'{RecessionName}: Feb 2020- Apr 2020:Gold')
recess_usd    =concat_df.loc[start_date:end_date,:].hvplot(y='usd_close'    ,height=300,width=600,ylabel='$-Index'                          ,title=f'{RecessionName}: Feb 2020- Apr 2020:USD Index')
recess_tech   =concat_df.loc[start_date:end_date,:].hvplot(y='tech_close'   ,height=300,width=600,ylabel='Technology' ,label='Technology'   ,title=f'{RecessionName}: Feb 2020- Apr 2020:Technology Sector')
recess_fin    =concat_df.loc[start_date:end_date,:].hvplot(y='fin_close'    ,height=300,width=600,ylabel='Financial'  ,label='Financial'    ,title=f'{RecessionName}: Feb 2020- Apr 2020:Financial Sector')
recess_engy   =concat_df.loc[start_date:end_date,:].hvplot(y='engy_close'   ,height=300,width=600,ylabel='Energy'     ,label='Energy'       ,title=f'{RecessionName}: Feb 2020- Apr 2020:Energy Sector')
recess_utils  =concat_df.loc[start_date:end_date,:].hvplot(y='utils_close'  ,height=300,width=600,ylabel='Utilities'  ,label='Utilities'    ,title=f'{RecessionName}: Feb 2020- Apr 2020:Utilities Sector')
recess_health =concat_df.loc[start_date:end_date,:].hvplot(y='health_close' ,height=300,width=600,ylabel='Health'     ,label='Health'       ,title=f'{RecessionName}: Feb 2020- Apr 2020:Health Sector')
recess_prop   =concat_df.loc[start_date:end_date,:].hvplot(y='prop_close'   ,height=300,width=600,ylabel='Real Estate',label='Real Estate'  ,title=f'{RecessionName}: Feb 2020- Apr 2020:Real Estate Sector')
recess_oil    =concat_df.loc[start_date:end_date,:].hvplot(y='oil_close'    ,height=300,width=600,ylabel='Oil'        ,label='Oil'          ,title=f'{RecessionName}: Feb 2020- Apr 2020:Oil Price')

sp500_vix     =concat_df.loc[start_date:end_date,:].hvplot.scatter(x='sp500_close',height=300,width=600 ,y='vix_close',xlabel='S & P 500', ylabel='CBOE Volatility Index')
sp500_tnx     =concat_df.loc[start_date:end_date,:].hvplot.scatter(x='sp500_close',height=300,width=600 ,y='tnx_close',xlabel='S & P 500', ylabel='Treasury Yield 10 years')
sp500_gold    =concat_df.loc[start_date:end_date,:].hvplot.scatter(x='sp500_close',height=300,width=600 ,y='gold_close',xlabel='S & P 500', ylabel='Gold Index')
sp500_usd     =concat_df.loc[start_date:end_date,:].hvplot.scatter(x='sp500_close',height=300,width=600 ,y='usd_close' ,xlabel='S & P 500', ylabel='USD Index')

display(sp500_vix + sp500_tnx + sp500_gold + sp500_usd)
(recess_tech * recess_fin * recess_engy * recess_utils * recess_health * recess_prop * recess_oil).opts(title=f'{RecessionName}: Feb 2020- Apr 2020: Sectorwise'
                                                                                                 ,ylabel='Index'
                                                                                                 ,legend_position='bottom'
                                                                                                 ,height=500,width=900)

:Layout
   .Scatter.I   :Scatter   [sp500_close]   (vix_close)
   .Scatter.II  :Scatter   [sp500_close]   (tnx_close)
   .Scatter.III :Scatter   [sp500_close]   (gold_close)
   .Scatter.IV  :Scatter   [sp500_close]   (usd_close)

:Overlay
   .Curve.Technology  :Curve   [Date]   (tech_close)
   .Curve.Financial   :Curve   [Date]   (fin_close)
   .Curve.Energy      :Curve   [Date]   (engy_close)
   .Curve.Utilities   :Curve   [Date]   (utils_close)
   .Curve.Health      :Curve   [Date]   (health_close)
   .Curve.Real_Estate :Curve   [Date]   (prop_close)
   .Curve.Oil         :Curve   [Date]   (oil_close)

In [23]:
start_date='03-01-2001'
end_date='11-30-2001'

recess_df=concat_df.loc[start_date:end_date,:]

Max=recess_df.max()
Min=recess_df.min()
MaxDrop=(Min-Max)/Max*100

MaxDropMatrix=pd.concat([Max,Min,MaxDrop],axis=1)

MaxDropMatrix=MaxDropMatrix.rename(columns={0:'High'
                                            ,1:'Low'
                                            ,2:'%Chg'})

MaxDropMatrix

,High,Low,%Chg
sp500_close,1312.830000,984.540000,-25.006284
vix_close,43.740002,18.760000,-57.110199
tnx_close,5.507000,4.201000,-23.715271
gold_close,293.250000,255.950000,-12.719523
usd_close,120.900002,111.169998,-8.047977
tech_close,23.450680,14.268366,-39.155854
fin_close,15.236972,11.670535,-23.406468
engy_close,19.679180,13.780057,-29.976468
utils_close,15.502624,12.885494,-16.881852
health_close,21.795509,15.605216,-28.401690


In [24]:
start_date='03-01-2001'
end_date='11-30-2001'

start_df=concat_df.loc[start_date,:]
endp_df=concat_df.loc[end_date,:]

chg_df=((endp_df-start_df)/start_df) *100

overall_chg=pd.concat([start_df,endp_df,chg_df],axis=1,join='inner')

overall_chg.columns.values[0]=f'as of {start_date}'
overall_chg.columns.values[1]=f'as of {end_date}'
overall_chg.columns.values[2]=f'Pct Chg'

overall_chg

,as of 03-01-2001,as of 11-30-2001,Pct Chg
sp500_close,1241.230000,1139.450000,-8.199931
vix_close,28.080000,23.840000,-15.099715
tnx_close,4.856000,4.744000,-2.306425
gold_close,266.350000,275.500000,3.435329
usd_close,111.279999,116.129997,4.358374
tech_close,21.927896,18.684399,-14.791647
fin_close,14.249502,13.589666,-4.630590
engy_close,18.202068,14.573072,-19.937273
utils_close,14.564467,12.885494,-11.527871
health_close,20.338774,18.790976,-7.610085


In [25]:
start_date='02-28-2001'
end_date='11-30-2007'

initial_df=concat_df.loc[start_date,:]


PostRec_df=concat_df.loc[end_date:,:]
PostRec_df=PostRec_df.reset_index()


idx_list={ 0:'sp500_close'
          ,1:'vix_close'
          ,2:'tnx_close'
          ,3:'gold_close'
          ,4:'usd_close'
          ,5:'tech_close'
          ,6:'fin_close'
          ,7:'engy_close'
          ,8:'utils_close'
          ,9:'health_close'
          ,10:'prop_close'
          ,11:'oil_close'
         }
          

results=[]
for key,value in idx_list.items():
    days_to_rec_df1=PostRec_df[['Date',value]].loc[PostRec_df[value]>=initial_df[key],:]
    # display(days_to_rec_df1)
    
    if days_to_rec_df1.empty != True:
        days_to_rec_df1=days_to_rec_df1.iloc[0]
        days_to_rec_df1=list(PostRec_df[['Date',value]].loc[PostRec_df[value]>=initial_df[key],:].iloc[0,])
        days_to_rec_df1.insert(0, value)
        # print(days_to_rec_df1)
        results.append(days_to_rec_df1)
    else:
        days_to_rec_df1=[value,pd.to_datetime(start_date),0]
        results.append(days_to_rec_df1)

results_df=pd.DataFrame(results)

results_df
results_df=results_df.rename(columns={0:"Name",
                                      1:"Date of Recovery",
                                      2:"Value at Recovery date"})

results_df['Num of Days to Recovery']=results_df["Date of Recovery"]-pd.to_datetime(start_date)
results_df['Num of Months to Recovery']=results_df['Num of Days to Recovery']/np.timedelta64(1, 'M')
results_df['Num of Months to Recovery']=results_df['Num of Months to Recovery'].astype(int)
results_df=results_df.set_index('Name')
results_df

,Date of Recovery,Value at Recovery date,Num of Days to Recovery,Num of Months to Recovery
Name,,,,
sp500_close,2007-11-30,1481.140000,2466 days,81
vix_close,2001-02-28,0.000000,0 days,0
tnx_close,2001-02-28,0.000000,0 days,0
gold_close,2007-11-30,783.500000,2466 days,81
usd_close,2001-02-28,0.000000,0 days,0
tech_close,2001-02-28,0.000000,0 days,0
fin_close,2007-11-30,18.518604,2466 days,81
engy_close,2007-11-30,46.343433,2466 days,81
utils_close,2007-11-30,24.710802,2466 days,81
